# Kompresja tekstu

In [1]:
from queue import PriorityQueue
from queue import LifoQueue
from collections import defaultdict
from time import time
import numpy as np
import pandas as pd
import os

### Statyczny algorytm Huffmana

Klasa reprezentująca węzeł w drzewie Huffmana.

In [2]:
class Node:
    def __init__(self,weight, left=None, right=None, char=None):
        self.weight=weight
        self.left=left
        self.right=right
        self.char=char

Funkcja zliczająca litery w tekście. Zwraca słownik {litera: ilość wystąpień}

In [3]:
def count_letters(text):
    letter_counts=dict()
    for letter in text:
        if not letter in letter_counts:
            letter_counts[letter]=0
        else:
            letter_counts[letter]+=1
    return letter_counts

Funkcja budująca drzewo Huffmana na podstawie słownika zawierającego zliczone litery.

In [4]:
def static_huffman_tree(letter_counts):
    nodes=PriorityQueue()
    for a, weight in letter_counts.items():
        nodes.put((weight, a, Node(weight, char=a)))
    while(nodes.qsize()>1):
        element_1=nodes.get()
        element_1, a= element_1[2],element_1[1]
        element_2=nodes.get()[2]
        nodes.put((element_1.weight+element_2.weight, a,Node(element_1.weight+element_2.weight, left=element_1, right=element_2)))
    return nodes.get()[2] 

Funkcja zwracająca słownik {litera: kod Huffmana} na podstawie drzewa Huffmana.

In [5]:
def static_huffman_code(huffman_tree):
    stack=LifoQueue()
    codes=dict()
    stack.put(("", huffman_tree))
    while(not stack.empty()):
        code, node=stack.get()
        if node.char is not None:
            codes[node.char]=code
        else:
            if node.left is not None:
                stack.put((code+"0", node.left))
            if node.right is not None:
                stack.put((code+"1", node.right))
    return codes

Funkcja (rekurencyjna) zwracająca drzewo Huffmana zapisane jako ciąg zer i jedynek.\
\
Sposób kodowania drzewa:
* Stosujemy post-order traversal w przechodzeniu po drzewie.
* Jeśli jesteśmy w węźle niebędącym liściem: kod lewego poddrzewa + kod prawego poddrzewa + '0'
* Jeżeli jesteśmy w liściu: '1' + 8 bitowy kod znaku znajdującego się w liściu.

In [6]:
def tree_to_binary(node):
    if node is None: return ""
    elif node.char is not None:
        char_bin=str(bin(node.char))[2:]
        #char_bin=str(bin(ord(node.char)))[2:]
        return "1"+"0"*(8-len(char_bin))+char_bin
    else:
        return tree_to_binary(node.left)+tree_to_binary(node.right)+"0"

Funkcja kompresująca plik i tworząca plik binarny o nazwie output_filename.\
\
Plik skompresowany ma postać:
* Zakodowane drzewo Huffmana (niezbędne do dekompresji)
* Jedno dodatkowe '0' sygnalizujące koniec bitów kodujących drzewo
* Zakodowana zawartość pliku\
\
Na koniec zawartości pliku dodany został znak '$', który sygnalizuje koniec pliku. Dzięki temu przy dekompresji nie uzyskamy nadmiarowych danych.

In [7]:
def static_huffman_encoding(text):
    # $ będzie oznaczać koniec pliku, aby przy dekompresji nie uzyskac nadmiarowych danych
    text.append(ord('$'))
    h_tree=static_huffman_tree(count_letters(text))
    h_code=static_huffman_code(h_tree)
    encoded=tree_to_binary(h_tree)+"0"
    for letter in text:
        encoded=encoded+h_code[letter]
    
    buff=0
    counter=0
    return encoded

In [8]:
def static_huffman_encoding_files(input_file, output_filename):
    with open(input_file, "rb") as f:
        text = bytearray(f.read())
    start=time()
    encoded=static_huffman_encoding(text)
    end=time()
    buff=0
    counter=0
    with open(output_filename+".bin", "wb") as f:
        for bit in encoded:
            buff=(buff << 1) | int(bit)
            counter+=1
            if counter == 8:
                f.write(bytes([buff]))
                buff=0
                counter=0
        if counter>0:
            buff= buff << (8-counter)
            f.write(bytes([buff])) 
    return end-start

Funkcja dekompresująca plik z pliku binarnego.

In [9]:
def static_huffman_decoding(bin_str):
    #rekonstrukcja drzewa
    h_tree=None
    q=LifoQueue()
    i=0
    while(True):
        if bin_str[i]=="1":
            char=bin_str[i+1:i+9]
            q.put(Node(0,char=int(char, 2)))
            #q.put(Node(0,char=chr(int(char, 2))))
            i+=9
        else:
            i+=1
            right=q.get()
            if(q.empty()): 
                h_tree=right
                break
            left=q.get()
            q.put(Node(0, left=left, right=right))   
    #dekodowanie pliku
    decoded=bytearray()
    curr_node=h_tree
    while i<len(bin_str):
        if bin_str[i]=='0':
            curr_node=curr_node.left
        elif bin_str[i]=='1':
            curr_node=curr_node.right
        if curr_node.char is not None:
            if curr_node.char==ord('$'): break
            decoded.append(curr_node.char)
            curr_node=h_tree
        i+=1
    
    return decoded

In [10]:
def static_huffman_decoding_files(input_file, output_file):
    with open(input_file, 'rb') as f:
        buff = bytearray(f.read())

    bin_str=''.join(f'{byte:08b}' for byte in buff)
    
    start=time()
    decoded=static_huffman_decoding(bin_str)
    end=time()

    with open(output_file, "wb") as f:
        text = f.write(decoded)
    return end-start

### Pliki testujące

Wybrano: Iliadę Homera (w formacie txt) z projektu Gutenberg, plik inode.c z kodem źródłowym jądra Linuxa oraz wylosowano (poniżej) plik ze znakami losowanymi z rozkładu jednostajnego - uwzględniono 255 wartości poza wartością '$', gdyż znak ten na samym początku przyjęto za oznaczenie końca pliku.

Iliadę i inode.c przycięto do rozmiarów 1kB, 10kB, 100kB, 1MB wykorzystując funkcje systemowe lub powielono treść tak by rozmiary były dokładne.

#### Generowanie plików z losowymi znakami

In [11]:
def generate_random_file(filename, filesize): #filesize w bajtach
    letters=np.random.randint(1,256,filesize)
    #usuniecie $
    mask= letters == ord('$')
    letters[mask]=0
    with open(filename, 'wb') as f:
        for letter in letters:
            f.write(bytes([letter]))

In [12]:
generate_random_file('text_files/random1kB.txt', 1024)
generate_random_file('text_files/random10kB.txt', 10240)
generate_random_file('text_files/random100kB.txt', 102400)
generate_random_file('text_files/random1MB.txt', 1048576)

### Kompresja i dekompresja z pomiarem czasu

#### Kompresja plików

In [13]:
en_time_r1kB=static_huffman_encoding_files('text_files/random1kB.txt', 'compressed/random1kB')
en_time_r10kB=static_huffman_encoding_files('text_files/random10kB.txt', 'compressed/random10kB')
en_time_r100kB=static_huffman_encoding_files('text_files/random100kB.txt', 'compressed/random100kB')
en_time_r1MB=static_huffman_encoding_files('text_files/random1MB.txt', 'compressed/random1MB')

In [14]:
en_time_il1kB=static_huffman_encoding_files('text_files/iliada1kB.txt', 'compressed/iliada1kB')
en_time_il10kB=static_huffman_encoding_files('text_files/iliada10kB.txt', 'compressed/iliada10kB')
en_time_il100kB=static_huffman_encoding_files('text_files/iliada100kB.txt', 'compressed/iliada100kB')
en_time_il1MB=static_huffman_encoding_files('text_files/iliada1MB.txt', 'compressed/iliada1MB')

In [15]:
en_time_in1kB=static_huffman_encoding_files('text_files/inode1kB.c', 'compressed/inode1kB')
en_time_in10kB=static_huffman_encoding_files('text_files/inode10kB.c', 'compressed/inode10kB')
en_time_in100kB=static_huffman_encoding_files('text_files/inode100kB.c', 'compressed/inode100kB')
en_time_in1MB=static_huffman_encoding_files('text_files/inode1MB.c', 'compressed/inode1MB')

#### Dekompresja plików

In [16]:
de_time_r1kB=static_huffman_decoding_files('compressed/random1kB.bin', 'decompressed/d_random1kB.txt')
de_time_r10kB=static_huffman_decoding_files('compressed/random10kB.bin', 'decompressed/d_random10kB.txt')
de_time_r100kB=static_huffman_decoding_files('compressed/random100kB.bin', 'decompressed/d_random100kB.txt')
de_time_r1MB=static_huffman_decoding_files('compressed/random1MB.bin', 'decompressed/d_random1MB.txt')

In [17]:
de_time_il1kB=static_huffman_decoding_files('compressed/iliada1kB.bin', 'decompressed/d_iliada1kB.txt')
de_time_il10kB=static_huffman_decoding_files('compressed/iliada10kB.bin', 'decompressed/d_iliada10kB.txt')
de_time_il100kB=static_huffman_decoding_files('compressed/iliada100kB.bin', 'decompressed/d_iliada100kB.txt')
de_time_il1MB=static_huffman_decoding_files('compressed/iliada1MB.bin', 'decompressed/d_iliada1MB.txt')

In [18]:
de_time_in1kB=static_huffman_decoding_files('compressed/inode1kB.bin', 'decompressed/d_inode1kB.c')
de_time_in10kB=static_huffman_decoding_files('compressed/inode10kB.bin', 'decompressed/d_inode10kB.c')
de_time_in100kB=static_huffman_decoding_files('compressed/inode100kB.bin', 'decompressed/d_inode100kB.c')
de_time_in1MB=static_huffman_decoding_files('compressed/inode1MB.bin', 'decompressed/d_inode1MB.c')

### Czasy kompresji i dekompresji

In [19]:
ixs=["inode.c", "iliada.txt", "random.txt"]
data_entimes={'file': ixs,
      '1kB': [en_time_in1kB,en_time_il1kB,en_time_r1kB],
      '10kB': [en_time_in10kB,en_time_il10kB,en_time_r10kB],
      '100kB': [en_time_in100kB,en_time_il100kB,en_time_r100kB],
      '1MB': [en_time_in1MB,en_time_il1MB,en_time_r1MB]}

data_frame_entimes=pd.DataFrame(data_entimes)
t_entimes=data_frame_entimes.style.set_caption("Czasy kompresji")
display(t_entimes)

,file,1kB,10kB,100kB,1MB
0,inode.c,0.002000,0.004000,0.027005,0.945213
1,iliada.txt,0.000999,0.004138,0.028007,0.841188
2,random.txt,0.003990,0.007001,0.029006,1.987463


In [20]:
data_detimes={'file': ixs,
      '1kB': [de_time_in1kB,de_time_il1kB,de_time_r1kB],
      '10kB': [de_time_in10kB,de_time_il10kB,de_time_r10kB],
      '100kB': [de_time_in100kB,de_time_il100kB,de_time_r100kB],
      '1MB': [de_time_in1MB,de_time_il1MB,de_time_r1MB]}

data_frame_detimes=pd.DataFrame(data_detimes)
t_detimes=data_frame_detimes.style.set_caption("Czasy dekompresji")
display(t_detimes)

,file,1kB,10kB,100kB,1MB
0,inode.c,0.001000,0.014003,0.129030,1.305627
1,iliada.txt,0.001999,0.014004,0.120027,1.213304
2,random.txt,0.004000,0.022005,0.190042,1.910953


### Współczynniki kompresji

In [21]:
def comp_rate(raw_name, compressed_name):
    raw_stat=os.stat(raw_name)
    comp_stat=os.stat(compressed_name)
    return 1-comp_stat.st_size/raw_stat.st_size

In [22]:
kB1=[comp_rate('text_files/inode1kB.c', 'compressed/inode1kB.bin'),
    comp_rate('text_files/iliada1kB.txt', 'compressed/iliada1kB.bin'),
    comp_rate('text_files/random1kB.txt', 'compressed/random1kB.bin')]

kB10=[comp_rate('text_files/inode10kB.c', 'compressed/inode10kB.bin'),
    comp_rate('text_files/iliada10kB.txt', 'compressed/iliada10kB.bin'),
    comp_rate('text_files/random10kB.txt', 'compressed/random10kB.bin')]

kB100=[comp_rate('text_files/inode100kB.c', 'compressed/inode100kB.bin'),
    comp_rate('text_files/iliada100kB.txt', 'compressed/iliada100kB.bin'),
    comp_rate('text_files/random100kB.txt', 'compressed/random100kB.bin')]

MB1=[comp_rate('text_files/inode1MB.c', 'compressed/inode1MB.bin'),
    comp_rate('text_files/iliada1MB.txt', 'compressed/iliada1MB.bin'),
    comp_rate('text_files/random1MB.txt', 'compressed/random1MB.bin')]

data={'file': ixs,
      '1kB': kB1,
      '10kB': kB10,
      '100kB': kB100,
      '1MB': MB1}

data_frame=pd.DataFrame(data)
t=data_frame.style.set_caption("Współczynniki kompresji w procentach")
display(t)

,file,1kB,10kB,100kB,1MB
0,inode.c,0.329102,0.341406,0.356455,0.355742
1,iliada.txt,0.272461,0.351465,0.421289,0.406939
2,random.txt,-0.306641,-0.030957,-0.003008,-0.000262


Dzień dobry, 
\
oto moja realizacja zadania o kompresji tekstu.\
Niestety nie udało mi się znaleźć materiałów (w zeszłorocznym wykładzie ani w Internecie), które pomogłyby mi zrozumieć algorytm dynamicznego kodowania Huffmana na tyle by go zaimplementować. Na dodatek bardzo dużo czasu poświęciłam na walkę z Pythonem aby w odpowiedni sposób wczytywał i zapisywał pliki (szczególnie z uwzględnieniem znaków niedrukowalnych).
\
Na posypanie solą ran powyższe współczynniki kompresji wyszły ujemne dla pliku z losowymi znakami! Myślę, że jest to spowodowane tym, że dodatkową pamięć w pliku skompresowanym zajmuje zapisanie drzewa dekompresji, które w przypadku tych plików jest zapewne całkiem sporych rozmiarów w porównaniu do reszty pliku. O poprawności mojego wnioskowania świadczy fakt, że współczynnik kompresji dla 1MB pliku random.txt jest większy (mimo że wciąż ujemny) niż dla plików mniejszych. \
\
Ola Smela